# imports 

In [1]:
! pip install ucimlrepo

In [2]:
import numpy as np 
import torch
from ucimlrepo import fetch_ucirepo
import pandas as pd

In [3]:
import os
#from tensorflow.keras import Sequential # type: ignore
#from tensorflow.keras.optimizers import Adam # type: ignore
#from tensorflow.keras.layers import( # type: ignore
#    InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout,GlobalAveragePooling2D
#)
os.environ["KERAS_BACKEND"] = "torch"
import keras

# Code dataloader pytorch

In [4]:

class DryBeamDataset(torch.utils.data.Dataset):

    def __init__(self):
        dry_bean = fetch_ucirepo(id=602)
        dffeatures = dry_bean.data.features
        dffeatures= (dffeatures-dffeatures.mean())/dffeatures.std()
        self.X = dffeatures.values.astype(np.float32)
        cat = pd.Categorical(dry_bean.data.targets['Class'])
        self.y = cat.codes.astype(int)
        self.nsamples = self.y.shape[0]
        self.nclasses = dry_bean.data.targets['Class'].unique().shape[0]
        print(f'DryBeamDataset has {self.nsamples} of {self.nclasses} classes.')

    def __len__(self):
        return self.nsamples

    def __getitem__(self, idx):
        sample = (torch.from_numpy(self.X[idx,:]),
        torch.tensor(self.y[idx]))
        return sample

In [5]:
def train_test_split_pytorch(batch_size:int = 16, p_train:float = 0.7, p_test:float = 0.3):
    dbeamdset = DryBeamDataset() 

    generator1 = torch.Generator().manual_seed(42)
    traindset,testdset = torch.utils.data.random_split(dbeamdset, [p_train, p_test],
    generator=generator1)

    traindataloader = torch.utils.data.DataLoader(traindset, batch_size=batch_size,
    shuffle=True)
    testdataloader = torch.utils.data.DataLoader(testdset, batch_size=batch_size,
    shuffle=True)
    return traindataloader, testdataloader

# Rede neural utilizando keras

In [4]:
def keras_model(num_classes:int = 7, input_shape:tuple = (16,), verbose = True):
    """
    Utilizando o mesmo modelo que foi apresentado nos slides do professor
    """

    model = keras.Sequential(
        [
            keras.layers.InputLayer(shape = input_shape),
            keras.layers.Dense(512),
            keras.layers.ReLU(),
            keras.layers.Dense(128),
            keras.layers.ReLU(),
            keras.layers.Dense(num_classes),
            
        ])
    if verbose is True:
        model.summary()
    
    return model

# Trainando o modelo utilizano o dataloader do pytorch

In [19]:
def train_dataloader_pytorch(batch_size:int = 16, epochs:int = 30, lr:float = 1e-3):
    
    train, test = train_test_split_pytorch(batch_size = batch_size)
    model = keras_model(verbose = False)
    
    model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=lr),
                  metrics = [
                      keras.metrics.SparseCategoricalAccuracy(name="acc"),
                  ])
    
    hist = model.fit(train,
              validation_data = test, 
              batch_size = batch_size, 
              verbose = True, 
              epochs = epochs)
    
    return hist

In [20]:
hist = train_dataloader_pytorch()

DryBeamDataset has 13611 of 7 classes.
Epoch 1/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.2796 - loss: 9.5731 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 2/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4037 - loss: 8.8287 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 3/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4075 - loss: 8.8117 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 4/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4110 - loss: 8.9346 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 5/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - acc: 0.4116 - loss: 8.8982 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 6/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4095 - loss: 8.9065 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 7/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.3977 - loss: 8.6648 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 8/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4050 - loss: 8.9232 - val_acc: 0.3970 - val_loss: 8.6018
Epoch 9/30
596/59

## Comentarios

#### Obtive uma acuracia final de aprocimadamente 40%, fazendo um split de validação e otimizando os hiperparêmetros, acredito que o resultado possa melhorar muito. Porêm o objetivo desta tarefa não é esse, por isso vou seguir para a implementação da mesma rede neural mas sem o dataloader do pytorch.

# Treinando com a api KERAS sem utilizar o dataloader do pytorch

In [8]:
def keras_train(batch_size:int = 16, epochs:int = 30, lr:float = 1e-4):
   
    # Carregamento e pré-processamento do dataset
    dry_bean = fetch_ucirepo(id=602)
    dffeatures = dry_bean.data.features
    dffeatures= (dffeatures-dffeatures.mean())/dffeatures.std()
    X = dffeatures.values.astype(np.float32)
    cat = pd.Categorical(dry_bean.data.targets['Class'])
    y = cat.codes.astype(int)
    
    nsamples = y.shape[0]
    nclasses = dry_bean.data.targets['Class'].unique().shape[0]
    print(f'DryBeamDataset has {nsamples} of {nclasses} classes.')

    # Divide em treino (70%) e teste (30%) com shuffle
    np.random.seed(42)
    indices = np.random.permutation(nsamples)
    train_size = int(0.7 * nsamples)
    train_idx = indices[:train_size]
    test_idx = indices[train_size:]

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = keras_model()

    model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=lr),
                  metrics = [
                      keras.metrics.SparseCategoricalAccuracy(name="acc"),
                  ])
    
    hist = model.fit(X_train, y_train,
              validation_data =(X_test, y_test), 
              batch_size = batch_size, 
              verbose = True, 
              epochs = epochs)
    
    return hist


In [9]:
hist = keras_train()

DryBeamDataset has 13611 of 7 classes.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 512)                 │           8,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_4 (ReLU)                       │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_5 (ReLU)                       │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 75,271 (294.03 KB)

 Trainable params: 75,271 (294.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - acc: 0.3815 - loss: 6.9187 - val_acc: 0.4863 - val_loss: 6.7750
Epoch 2/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.3621 - loss: 8.0765 - val_acc: 0.3151 - val_loss: 12.0467
Epoch 3/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - acc: 0.3306 - loss: 11.3804 - val_acc: 0.4354 - val_loss: 9.1263
Epoch 4/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4352 - loss: 7.7558 - val_acc: 0.4711 - val_loss: 5.9195
Epoch 5/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4597 - loss: 7.4074 - val_acc: 0.4748 - val_loss: 8.4767
Epoch 6/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4159 - loss: 7.7648 - val_acc: 0.4145 - val_loss: 6.7535
Epoch 7/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - acc: 0.4143 - loss: 4.9748 - val_acc: 0.3979 - val_loss: 5.4327
Epoch 8/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - acc: 0.3884 - loss: 6.0547 - val_acc: 0.3805 - val_loss: 6.0329
Epoch 9/30
596/596 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - 